## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

Входные данные — статистика о платёжеспособности клиентов, содержит:

- children — количество детей в семье
- days_employed — трудовой стаж в днях
- dob_days — возраст клиента в годах
- education — образование клиента
- education_id — идентификатор образования
- family_status — семейное положение
- family_status_id — идентификатор семейного положения
- gender — пол клиента
- income_type — тип занятости
- debt — имел ли задолженность по возврату кредитов
- total_income — доход в месяц
- purpose — цель получения кредита

Плат работы:
1. Импорт файла с данными, изучение общей информации.
2. Предобработка данных
    - Обработка пропусков
    - Замена типа данных
    - Обработка дубликатов
    - Лемматизация
    - Категоризация данных
3. Ответы на вопросы
4. Общий вывод

### 1. Импорт файла с данными, изучение общей информации. 

In [1]:
import pandas as pd #импорт pandas
from IPython.display import display

In [2]:
clients = pd.read_csv('/datasets/data.csv') #считывание файла с данными

clients.info() #инфо об инпортированном файле
clients.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


### Вывод

Таблица состоит из 12-ти столбцов, из которых: 2 имеют тип данных float64, 5 - int64, 5- object. В столбцах `days_employed` и `total_income` имеются пропущенные значения. Скорее всего, это связано с технической ошибкой при выгрузке данных. Столбец `days_employed` можно исклютить из анализа по принине отсутствия каких - либо вопросов, связанных с смысловой нагрузкой столбца. 

### 2. Предобработка данных

### Обработка пропусков

In [3]:
# заполнение пропусков в 'total_income' медианным значением в соответствии с типом занятоски
#print(clients['total_income'].isna().sum()) # подсчет пропусков в 'total_income' до обработки
for income_type in clients['income_type'].unique(): # для всех уникальных значений 'income_type'
    median = clients.loc[clients['income_type'] == income_type, 'total_income'].median() # рассчитываем медиану в 'total_income'
    print(income_type, median) # вывод соответствия категорий из 'income_type' медиане по 'total_income'
    # заполняем медианным значением пропуски в 'total_income' в соответствии с категориями в 'income_type'
    clients.loc[(clients['total_income'].isna()) & (clients['income_type'] == income_type), 'total_income'] = median 
#print(clients['total_income'].isna().sum()) # подсчет пропусков в 'total_income' после обработки

clients.loc[clients['children'] == 20, 'children'] = 2 # в столбце 'children' преобразовываем лишние значение "20" в "2"
clients.loc[clients['children'] == -1, 'children'] = 1 # в столбце 'children' преобразовываем лишние значение "-1" в "1"
dob_years_mean = clients['dob_years'].mean() # из-за присутствия в столбце 'dob_years' возраста 0 лет, 
# заменим его на средний по столбцу. Расчет среднего возраста в столбце 'dob_years'
dob_years_mean = dob_years_mean.astype(int) # т.к. возраст получился не "целый", переводим в соотв. тип
clients.loc[clients['dob_years'] == 0, 'dob_years'] = dob_years_mean # заменяем 0 лет средним значением
clients = clients.loc[clients['gender'] != 'XNA'] # удаляем строку с единственным неизв. значением в столбце 'gender'
#clients['gender'].value_counts() # проверка присутствия лишнего в столбце 'gender'
#clients['dob_years'].value_counts() # проверка присутствия лишнего в столбце 'dob_years'

сотрудник 142594.39684740017
пенсионер 118514.48641164352
компаньон 172357.95096577113
госслужащий 150447.9352830068
безработный 131339.7516762103
предприниматель 499163.1449470857
студент 98201.62531401133
в декрете 53829.13072905995


### Вывод

Есть пропуски в столбце `total_income`, их целесообразно заменить медианным значением столбца `total_income` используя группировку по параметру "тип занятости". Также убираем лишнее в столбце `children`, считая, что 20 детей имелось в ввиду 2, а -1 ребенок - 1. Одновременно с этим, в столбце `dob_years` присутствует возраст 0 лет, заменям на среднее по столбцу.

### Замена типа данных

In [4]:
clients['total_income'] = clients['total_income'].astype(int) # Приводим тип данных столбца 'total_income' к int64

### Вывод

Все столбцы, за исключением `total_income` имеют правильный тип. `total_income` переводим в целочисленный.

### Обработка дубликатов

In [5]:
clients['education'] = clients['education'].str.lower() # приводим все значения столбца 'education' к нижнему регистру
#clients['education'].value_counts() # проверяем удаление дубликатов в разных регистрах
#clients.duplicated().sum() # подсчет количества дубликатов в таблице
#clients[clients.duplicated(keep = False)].sort_values(by = clients.columns.values.tolist()) # печать дубликатов для анализа
clients = clients.drop_duplicates().reset_index(drop = True) # удаление дубликатов
clients.duplicated().sum() # проверка количества дубликатов после удаления

0

### Вывод

Дубликаты, присутствующие в таблице удалены. Дубликаты встречаются в столобце `education`. Их обработака ограничивается приведением значений столбца к нижнему регистру.

### Лемматизация

In [6]:
from pymystem3 import Mystem # импорт библиотеки
m = Mystem()
lemmas = m.lemmatize(' '.join(clients['purpose'])) # лемматизация столбца 'purpose'
from collections import Counter # импорт библиотеки
print(Counter(lemmas)) # определение наиболее популярных категорий


def pop_purpose(clients): # определение функции, позволяющей упростить столбец 'purpose'
  
    if 'жилье' in m.lemmatize(clients['purpose']): # объединение на укрупненную категорию 'недвижимость'
        return 'недвижимость'
    elif 'недвижимость' in m.lemmatize(clients['purpose']): # объединение на укрупненную категорию 'недвижимость'
        return 'недвижимость'
    elif 'автомобиль' in m.lemmatize(clients['purpose']): # объединение на укрупненную категорию 'автомобиль'
        return 'автомобиль'
    elif 'образование' in m.lemmatize(clients['purpose']): # объединение на укрупненную категорию 'образование'
        return 'образование'
    elif 'свадьба' in m.lemmatize(clients['purpose']): # объединение на укрупненную категорию 'свадьба'
        return 'свадьба'

    
clients['pop_purpose'] = clients.apply(pop_purpose, axis=1) # создание столбца в clients с упрощенным категориями
#clients.head(10)
#clients['pop_purpose'].value_counts() # проверка работы функции pop_purpose

Counter({' ': 55021, 'недвижимость': 6350, 'покупка': 5896, 'жилье': 4460, 'автомобиль': 4306, 'образование': 4013, 'с': 2918, 'операция': 2604, 'свадьба': 2324, 'свой': 2230, 'на': 2222, 'строительство': 1878, 'высокий': 1374, 'получение': 1314, 'коммерческий': 1311, 'для': 1289, 'жилой': 1230, 'сделка': 941, 'дополнительный': 906, 'заниматься': 904, 'подержать': 853, 'проведение': 768, 'сыграть': 765, 'сдача': 651, 'семья': 638, 'собственный': 635, 'со': 627, 'ремонт': 607, 'приобретение': 461, 'профильный': 436, 'подержанный': 111, '\n': 1})


### Вывод

Лемматизация словоформ в столбце `purpose` определила 4 наиболее часто встречающиеся леммы, на основе которых созданы 4 категории: "недвижимость, автомобиль, образование, свадьба". Также, описана функция `pop_purpose()`, позволяющая создать столбец с упращенными категорями в `clients`.

### Категоризация данных

In [7]:
#clients.groupby(['education_id', 'education']).agg({'total_income': 'count'})
#clients.groupby(['education_id', 'education']).agg({'debt': 'sum'})

# создание сводной табл. по категории "образование клиента"
education_dict = clients.pivot_table(index = ['education_id','education'], values = 'debt', aggfunc = ['count']) 
education_dict.columns = ['Кол-во заемщиков']
display(education_dict)

# создание сводной табл. по категории "семейное положение"
family_dict = clients.pivot_table(index = ['family_status_id', 'family_status'], values = 'debt', aggfunc = ['count'])
family_dict.columns = ['Кол-во заемщиков']
display(family_dict)

# создание сводной табл. по укрупненной категории "цель получения кредита"
purpose_dict = clients.pivot_table(index = ['pop_purpose'], values = 'debt', aggfunc = ['count'])
purpose_dict.columns = ['Кол-во заемщиков']
display(purpose_dict)

# создание сводной табл. по категории "количество детей в семье"
children_dict = clients.pivot_table(index = ['children'], values = 'debt', aggfunc = ['count'])
children_dict.columns = ['Кол-во заемщиков']
display(children_dict)

,,Кол-во заемщиков
education_id,education,
0,высшее,5250
1,среднее,15172
2,неоконченное высшее,743
3,начальное,282
4,ученая степень,6


,,Кол-во заемщиков
family_status_id,family_status,
0,женат / замужем,12339
1,гражданский брак,4150
2,вдовец / вдова,959
3,в разводе,1195
4,Не женат / не замужем,2810


,Кол-во заемщиков
pop_purpose,
автомобиль,4306
недвижимость,10810
образование,4013
свадьба,2324


,Кол-во заемщиков
children,
0,14090
1,4855
2,2128
3,330
4,41
5,9


### Вывод

Категоризация данных:
1. Расчет кол-ва заемщиков сгрупированных по категории "образование клиента"
Вывод: больше всего заемщиков среди людей, имеющих образование "среднее"

2. Расчет кол-ва заемщиков сгрупированных по категории "семейное положение"
Вывод: больше всего заемщиков среди людей, имеющих  семейное положение "женат / замужем"

3. Расчет кол-ва заемщиков сгрупированных по укрупненной категории "цель получения кредита"
Вывод: больше всего заемщиков среди людей, указавших цель получение кредита "недвижимость"

4. Расчет кол-ва заемщиков сгрупированных по категории "количество детей в семье"
Вывод: больше всего заемщиков среди людей, у которых детей нет

In [8]:
# разбивка клиентов на 4 категории по уровню дохода методом qcut
clients['income_group'] = pd.qcut(clients['total_income'], q = [0, 0.25, 0.5, 0.75, 1], 
                                  labels = ['очень низкий', 'низкий', 'средний', 'высокий'])
clients[['income_group', 'total_income']] # вывод результата
#clients.head()

,income_group,total_income
0,высокий,253875
1,низкий,112080
2,средний,145885
3,высокий,267628
4,средний,158616
...,...,...
21448,высокий,224791
21449,средний,155999
21450,очень низкий,89672
21451,высокий,244093


### 3. Ответы на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [116]:
# подсчет кол-ва заемщиков с долгом по столбцу 'children'
children_debt = clients.pivot_table(index = ['children'], values = 'debt', aggfunc = ['sum'])
children_debt.columns = ['Кол-во заемщиков c долгом']
#display(children_debt)
# подсчет процентного отношения кол-ва заемщиков с долгом относительно общего кол-ва замщиков

#children_debt_final = children_debt['Кол-во заемщиков c долгом'] / children_dict['Кол-во заемщиков']
#display(children_debt_final)

children_debt_final = children_debt.merge(children_dict, on='children', how='left')
children_debt_final['% невозврата'] = children_debt_final['Кол-во заемщиков c долгом'] / children_debt_final['Кол-во заемщиков']
children_debt_final.style.format({'% невозврата':'{:.2%}'})

,Кол-во заемщиков c долгом,Кол-во заемщиков,% невозврата
children,,,
0,1063,14090,7.54%
1,445,4855,9.17%
2,202,2128,9.49%
3,27,330,8.18%
4,4,41,9.76%
5,0,9,0.00%


### Вывод

Зависимость есть. У заемщиков, у которых нет детей, процент невозврата меньше, чем у заемщиков с детьми.
Исключение состваляют заемщики, у которых 5 детей.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [120]:
# подсчет кол-ва заемщиков с долгом по столбцу 'family_status'
family_debt = clients.pivot_table(index = ['family_status'], values = 'debt', aggfunc = ['sum'])
family_debt.columns = ['Кол-во заемщиков c долгом']
#display(family_debt)
# подсчет процентного отношения кол-ва заемщиков с долгом относительно общего кол-ва замщиков
#family_debt_final = (family_debt['Кол-во заемщиков c долгом'] / family_dict['Кол-во заемщиков']) * 100
#family_debt_final

family_debt_final = family_debt.merge(family_dict, on='family_status', how='left')
family_debt_final['% невозврата'] = family_debt_final['Кол-во заемщиков c долгом'] / family_debt_final['Кол-во заемщиков']
family_debt_final.style.format({'% невозврата':'{:.2%}'})

,Кол-во заемщиков c долгом,Кол-во заемщиков,% невозврата
family_status,,,
Не женат / не замужем,274,2810,9.75%
в разводе,85,1195,7.11%
вдовец / вдова,63,959,6.57%
гражданский брак,388,4150,9.35%
женат / замужем,931,12339,7.55%


### Вывод

Зависимость есть. Заемщики, состоящие в отношениях "Не женат / не замужем" и "гражданский брак" имеют больше процент задолженности по возврату кредита, чем заемщики состощие в "в разводе", "женат / замужем". Самые безопасные: "вдовец / вдова"

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [121]:
# подсчет кол-ва заемщиков с долгом по категориям столбца 'income_group'
income_group_debt = clients.pivot_table(index = ['income_group'], values = 'debt', aggfunc = ['sum'])
income_group_debt.columns = ['Кол-во заемщиков c долгом']
#income_group_debt
# подсчет общего количества заемщиков по категориям столбца 'income_group'
total_income_dict = clients.pivot_table(index = ['income_group'], values = 'total_income', aggfunc = ['count'])
total_income_dict.columns = ['Кол-во заемщиков']
#total_income_dict
# для категорий дохода: расчет процентного отношения кол-ва заемщиков с долгом относительно общего кол-ва заемщиков
#total_income_final = (income_group_debt['Кол-во заемщиков c долгом'] / total_income_dict['Кол-во заемщиков']) * 100
#total_income_final

total_income_final = income_group_debt.merge(total_income_dict, on='income_group', how='left')
total_income_final['% невозврата'] = total_income_final['Кол-во заемщиков c долгом'] / total_income_final['Кол-во заемщиков']
total_income_final.style.format({'% невозврата':'{:.2%}'})

,Кол-во заемщиков c долгом,Кол-во заемщиков,% невозврата
income_group,,,
очень низкий,427,5364,7.96%
низкий,483,5479,8.82%
средний,448,5247,8.54%
высокий,383,5363,7.14%


### Вывод

Зависимость есть. Заемщики с низким и средним уровнем дохода имеют больше просроченных задолженностей чем заемщики с очень низким. Заемщики с высоким уровнем дохода имеют количество просроченных задолженностей меньше остальных.

- Как разные цели кредита влияют на его возврат в срок?

In [124]:
pop_purpose_debt = clients.pivot_table(index = ['pop_purpose'], values = 'debt', aggfunc = ['sum'])
pop_purpose_debt.columns = ['Кол-во заемщиков c долгом']
#display(pop_purpose_debt)

#pop_purpose_final = (pop_purpose_debt['Кол-во заемщиков долговые'] / purpose_dict['Кол-во заемщиков']) * 100
#display(pop_purpose_final)

pop_purpose_final = pop_purpose_debt.merge(purpose_dict, on='pop_purpose', how='left')
pop_purpose_final['% невозврата'] = pop_purpose_final['Кол-во заемщиков c долгом'] / pop_purpose_final['Кол-во заемщиков']
pop_purpose_final.style.format({'% невозврата':'{:.2%}'})

,Кол-во заемщиков c долгом,Кол-во заемщиков,% невозврата
pop_purpose,,,
автомобиль,403,4306,9.36%
недвижимость,782,10810,7.23%
образование,370,4013,9.22%
свадьба,186,2324,8.00%


### Вывод

Из расчетов видно, что при оформлении кредита на "автомобиль" и "образование" риски формирования задолженности у клиентов больше, чем при оформлении кредита на "недвижимость" и "свадьбу"

### 4. Общий вывод

<h3>Исследование надёжности заёмщиков</h3>

Цель: построение модели кредитного скорингаю.

Входные данные — статистика о платёжеспособности клиентов, содержит:

- количество детей в семье
- трудовой стаж в днях
- возраст клиента в годах
- образование клиента
- идентификатор образования
- семейное положение
- идентификатор семейного положения
- пол клиента
- тип занятости
- имел ли задолженность по возврату кредитов
- доход в месяц
- цель получения кредита

Вопросы для исседования:

- Есть ли зависимость между наличием детей и возвратом кредита в срок?
- Есть ли зависимость между семейным положением и возвратом кредита в срок?
- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
- Как разные цели кредита влияют на его возврат в срок?



<h4>В результате исседования можно сформировать следующий вывод.</h4>

Для кредитной организации наиболее "безопасным" с точко зрения возврата кредита является клиент со следующими характеристиками. Заемщик является вдовцом или вдовой; не имеет детей; уровень дохода высокий; цель получение кредита: покупка недвижимости или проведение свадьбы.
Одновременно с этим, наиболее рискованным с точки зрения возврата кредита является заемщик со следующими характеристиками.
Заемщик не женат (не замужем) или состоит в гражданском браке; содержит 1, 2 или 4 детей; уровень дохода низкий или средний; цель получения кредита: автомобиль или образование.
